In [4]:
%pip install catboost

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
# import lightgbm as lgb
import xgboost as xgb



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [21]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [22]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [23]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [19]:
cols=X.columns
cols

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming X and X_testdata are already defined as DataFrames
# Show the columns of X
print("Columns of X before scaling:", X.columns)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_testdata_scaled = scaler.transform(X_testdata)

# Convert the scaled arrays back to DataFrames to retain column names
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_testdata_scaled_df = pd.DataFrame(X_testdata_scaled, columns=X_testdata.columns)

# Show the columns of the scaled DataFrame
print("Columns of X after scaling:", X_scaled_df.columns)

Columns of X before scaling: Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')
Columns of X after scaling: Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45'

In [26]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=2)

In [ ]:
from catboost import CatBoostClassifier

# Initialize the CatBoost classifier with default parameters
catboost_clf = CatBoostClassifier(
    iterations=5000,
    learning_rate=0.25,
    depth=1,
    random_seed=2,
    verbose=100,
   
)

# Fit the model to the training data
catboost_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = catboost_clf.predict(X_test)
y_pred_proba = catboost_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")

0:	learn: 0.2064908	total: 214ms	remaining: 17m 49s
100:	learn: 0.0120649	total: 1.96s	remaining: 1m 34s
200:	learn: 0.0112283	total: 3.15s	remaining: 1m 15s
300:	learn: 0.0108124	total: 4.42s	remaining: 1m 9s
400:	learn: 0.0105706	total: 5.63s	remaining: 1m 4s
500:	learn: 0.0103641	total: 6.84s	remaining: 1m 1s
600:	learn: 0.0102299	total: 8.08s	remaining: 59.1s
700:	learn: 0.0101171	total: 9.39s	remaining: 57.6s
800:	learn: 0.0100065	total: 10.9s	remaining: 57s
900:	learn: 0.0099114	total: 12.3s	remaining: 55.8s
1000:	learn: 0.0098272	total: 13.6s	remaining: 54.5s
1100:	learn: 0.0097447	total: 14.9s	remaining: 52.8s
1200:	learn: 0.0096724	total: 16.2s	remaining: 51.1s
1300:	learn: 0.0096003	total: 18s	remaining: 51s
1400:	learn: 0.0095397	total: 19.8s	remaining: 50.9s
1500:	learn: 0.0094766	total: 21.8s	remaining: 50.7s
1600:	learn: 0.0094192	total: 24.4s	remaining: 51.9s
1700:	learn: 0.0093605	total: 26.7s	remaining: 51.7s
1800:	learn: 0.0093148	total: 28.7s	remaining: 51s
1900:	lea

In [29]:
from sklearn.feature_selection import SelectFromModel
clf=catboost_clf
# Assuming clf is your trained model
selector = SelectFromModel(clf, max_features=60, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
X_testdata_selected = selector.transform(X_testdata)

# Get the selected feature names
selected_features = X_train.columns[selector.get_support()]

print(f"Selected 60 best features:")
print(selected_features)

Selected 60 best features:
Index(['X2', 'X9', 'X12', 'X14', 'X15', 'X17', 'X22', 'X26', 'X32', 'X38',
       'X39', 'X40', 'X42', 'X44', 'X58', 'X63', 'X66', 'X67', 'X69', 'X70',
       'X73'],
      dtype='object')


c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.metrics import roc_auc_score

# Select only the selected features from X_train and X_test
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Train the model with the selected features
clf.fit(X_train_selected, y_train)

# Predict the probabilities for X_test
y_pred_proba = clf.predict_proba(X_test_selected)[:, 1]

# Calculate the AUC value
auc_value = roc_auc_score(y_test, y_pred_proba)

# Print the AUC value
print(f'AUC: {auc_value}')

0:	learn: 0.1946057	total: 17.2ms	remaining: 1m 26s
100:	learn: 0.0123604	total: 1.21s	remaining: 58.8s
200:	learn: 0.0116317	total: 2.45s	remaining: 58.6s
300:	learn: 0.0113049	total: 3.49s	remaining: 54.4s
400:	learn: 0.0110975	total: 4.49s	remaining: 51.5s
500:	learn: 0.0109681	total: 5.55s	remaining: 49.8s
600:	learn: 0.0108721	total: 6.71s	remaining: 49.1s
700:	learn: 0.0107905	total: 7.83s	remaining: 48s
800:	learn: 0.0107232	total: 9.05s	remaining: 47.4s
900:	learn: 0.0106588	total: 10.3s	remaining: 46.8s
1000:	learn: 0.0106061	total: 11.6s	remaining: 46.2s
1100:	learn: 0.0105493	total: 12.7s	remaining: 44.9s
1200:	learn: 0.0105055	total: 13.8s	remaining: 43.5s
1300:	learn: 0.0104671	total: 14.9s	remaining: 42.4s
1400:	learn: 0.0104278	total: 16.1s	remaining: 41.5s
1500:	learn: 0.0103821	total: 17.5s	remaining: 40.9s
1600:	learn: 0.0103482	total: 19.1s	remaining: 40.6s
1700:	learn: 0.0103221	total: 20.8s	remaining: 40.3s
1800:	learn: 0.0102908	total: 22.3s	remaining: 39.6s
1900:

In [ ]:
# Select the features from the entire dataset
X_selected = X_scaled_df[selected_features]

# Train the model on the entire dataset with the selected features
clf.fit(X_selected, y)

# Make predictions on the entire dataset
y_pred = clf.predict(X_selected)
y_pred_proba = clf.predict_proba(X_selected)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
roc_auc = roc_auc_score(y, y_pred_proba)

print(f"Accuracy on entire dataset: {accuracy}")
print(f"ROC AUC Score on entire dataset: {roc_auc}")


0:	learn: 0.2079670	total: 17.4ms	remaining: 1m 26s
100:	learn: 0.0125643	total: 1.87s	remaining: 1m 30s
200:	learn: 0.0118266	total: 3.62s	remaining: 1m 26s
300:	learn: 0.0115373	total: 5.38s	remaining: 1m 24s
400:	learn: 0.0113500	total: 7.01s	remaining: 1m 20s
500:	learn: 0.0112121	total: 8.67s	remaining: 1m 17s
600:	learn: 0.0111088	total: 10.4s	remaining: 1m 16s
700:	learn: 0.0110150	total: 12.5s	remaining: 1m 16s
800:	learn: 0.0109496	total: 14.4s	remaining: 1m 15s
900:	learn: 0.0108958	total: 16.7s	remaining: 1m 15s
1000:	learn: 0.0108438	total: 18.9s	remaining: 1m 15s
1100:	learn: 0.0108048	total: 21.1s	remaining: 1m 14s
1200:	learn: 0.0107743	total: 23s	remaining: 1m 12s
1300:	learn: 0.0107391	total: 24.8s	remaining: 1m 10s
1400:	learn: 0.0107113	total: 26.6s	remaining: 1m 8s
1500:	learn: 0.0106852	total: 28.6s	remaining: 1m 6s
1600:	learn: 0.0106566	total: 30.9s	remaining: 1m 5s
1700:	learn: 0.0106284	total: 32.9s	remaining: 1m 3s
1800:	learn: 0.0106027	total: 34.9s	remaining

In [33]:
# Predict probabilities for the test set
X_testdata_scaled_df_selected=X_testdata_scaled_df[selected_features]
y_pred_proba_testdata = clf.predict_proba(X_testdata_scaled_df_selected)[:, 1]

y_pred_proba_testdata

array([9.67755180e-04, 1.71064126e-03, 2.22525939e-05, ...,
       8.58227380e-04, 7.54945418e-05, 8.43939392e-05])

In [36]:
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities_cat.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

test_set

Probabilities saved to test_set_with_probabilities.csv


,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y_probability
0,300001,79.000000,17.122318,0,0,1,170.200000,1,700,0,...,0.030000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000968
1,300002,38.000000,43.693579,0,0,1,165.100000,1,814,0,...,0.040000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.001711
2,300003,36.064225,23.998944,0,0,1,167.086735,1,662,0,...,0.006948,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000022
3,300004,61.846764,31.693449,0,3,1,182.355708,2,862,0,...,0.033153,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000218
4,300005,71.591991,20.086147,1,0,1,166.704917,2,335,0,...,0.004854,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105477,405478,35.781006,25.569364,0,0,1,158.542014,1,238,0,...,0.002452,0.0,0.0,0.000000,0.12259,0.0,0.0,0.0,0.0,0.000008
105478,405479,73.000000,30.849795,0,0,0,182.900000,1,114,0,...,0.730000,0.0,0.0,1.000000,0.00000,0.0,0.0,0.0,0.0,0.435084
105479,405480,61.872510,31.609595,0,4,1,155.783272,1,536,0,...,0.079609,0.0,0.0,0.826809,0.00000,0.0,0.0,0.0,0.0,0.000858
105480,405481,58.722824,34.631254,1,0,1,174.474165,2,517,0,...,0.000602,0.0,0.0,0.969882,0.00000,0.0,0.0,0.0,0.0,0.000075


In [37]:
import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities_cat.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

[9.67755180e-04 1.71064126e-03 2.22525939e-05 ... 4.35083935e-01
 8.58227380e-04 7.54945418e-05]
        RecordId  Y_probability
0         300001       0.000968
1         300002       0.001711
2         300003       0.000022
3         300004       0.000218
4         300005       0.000125
...          ...            ...
105477    405478       0.000008
105478    405479       0.435084
105479    405480       0.000858
105480    405481       0.000075
105481    405482       0.000084

[105482 rows x 2 columns]
